# OpenAlex Colombia
[OpenAlex](https://openalex.org/)  is an index of hundreds of millions of metadata of scientific articles in all areas.

We will use the  access via a API to obtain the metadata about the scientific articles of Colombia

Example of use: https://sds-investigacion.azurewebsites.net/app

In [8]:
import requests
import time
import json
import pandas as pd
from IPython.display import JSON

Below is an example of a JSON of the `work` entity. See the full documentation here: https://docs.openalex.org/

In [258]:
f=open('sample.txt','r')
d=f.read()
f.close()
JSON(eval(d))

<IPython.core.display.JSON object>

## API usage example
To use directly the obtained JSON, see next cell

In [2]:
co=[]
i=0
count=0
baseurl='https://api.openalex.org/works?filter=authorships.institutions.country_code:co&mailto=restrepo@udea.edu.co&per-page=200'
nc='*'

In [113]:
while nc:
    i+=1
    url=baseurl+f'&cursor={nc}'
    try:
        r=requests.get(url)
        cotmp=r.json().get('results')
    except:
        print('ERROR: bad url')
        break
    if i==1:
        try:
            count=r.json().get('meta').get('count')
        except:
            print('ERROR: bad count')
            break
    kk=[d.pop('abstract_inverted_index') for d in cotmp if d.get('abstract_inverted_index')]
    try:
        nc=r.json().get('meta').get('next_cursor')
    except:
        print('ERROR: bad cursor')
        break
    #progress
    print(f'{i} → {200*i}/{count}',end='\r')
    #Append to dictionary list (WARNING: HIG RAM USAGE)
    for p in cotmp:
        co.append(p)
    #=== backup =====
    f=open('openalexco.jsonl','a')
    for p in cotmp:
        json.dump(p,f)
        f.write("\n")
    f.close()
    #API limits
    time.sleep(0.1)
    #EMERGENGY EXIT
    if i*200>count:
        break

In [40]:
f=open('openalexco.json','w')
json.dump(co,f)
f.close()

In [116]:
f=open('openalexco.jsonl','a')
for p in co:
    json.dump(p,f)
    f.write("\n")
f.close()

## Use downloaded JSON

In [42]:
f=open('openalexco.json','r')
co=json.load(f)
f.close()

Medicine & Psychology Colombia

In [137]:
md=[d for d in co if len( {'Medicine','Psychology'}.intersection( 
       [ c.get('display_name') for c in d.get('concepts') if c.get('level')==0 ] 
          ) )>0 ]

In [153]:
len(md)

77352

In [216]:
f=open('openalexco_md.json','w')
json.dump(md,f)
f.close()

In [2]:
f=open('openalexco_md.json','r')
md=json.load(f)
f.close()

Medicine & Psychology Bogotá

In [3]:
insts=set()
for d in md:
    for a in d.get('authorships'):
        if a.get('institutions'):
            for i in a.get('institutions'):
                if i.get('country_code')=="CO":
                    insts.add(i.get('display_name'))

In [4]:
len(insts)

309

In [5]:
[i for i in insts if i.find('National University of Colombia')>-1]

['National University of Colombia at Medellín',
 'National University of Colombia at Manizales',
 'National University of Colombia']

In [6]:
[i for i in insts if i.find('Jav')>-1]

['Pontificia Universidad Javeriana']

In [7]:
insts_bg={'El Bosque University',
'Pontificia Universidad Javeriana',          
'National University of Colombia',
'Universidad Antonio Nariño',
'Universidad Externado de Colombia',
'Universidad de Bogotá Jorge Tadeo Lozano',
 'Universidad de Los Andes',
 'Universidad del Rosario',
 'University of La Sabana',
 'University of La Salle'}

In [8]:
mdbg=[]
for d in md:
    l=[ [i.get('display_name') for i in a.get('institutions')] for a in d.get('authorships') if a.get('institutions')]
    if len({item for sublist in l for item in sublist}.intersection(insts_bg))>0:
        mdbg.append(d)

In [9]:
len(mdbg)

23920

In [5]:
f=open('openalexco_mdbg.json','w')
json.dump(mdbg,f)
f.close()

NameError: name 'mdbg' is not defined

In [218]:
d=co[0]

Search for author

In [43]:
author='Diego Restrepo'
dr=[d for d in co if author in [ a.get('author').get('display_name') for a in d.get('authorships')]]

In [44]:
len(dr)

61

In [45]:
pd.DataFrame(dr).authorships.apply(lambda l: [a.get('author').get('id') for a in l if a.get('author').get('display_name')=='Diego Restrepo']
                                     ).str[0].unique()

array(['https://openalex.org/A4289616623',
       'https://openalex.org/A2110103959',
       'https://openalex.org/A4224864514',
       'https://openalex.org/A2306595523',
       'https://openalex.org/A4288869362',
       'https://openalex.org/A4290241673',
       'https://openalex.org/A2222954481'], dtype=object)

Search for Author ID

In [46]:
oa_id='https://openalex.org/A2306595523'
DR=[d for d in co if oa_id in [ a.get('author').get('id') for a in d.get('authorships')]]

In [47]:
len(DR)

43

In [48]:
pd.set_option('display.max_colwidth',200)
pd.DataFrame(DR)[['id','doi','title']].sort_values('title').drop_duplicates(subset=['title']).reset_index(drop=True)

,id,doi,title
0,https://openalex.org/W2194147853,https://doi.org/10.1016/j.nuclphysbps.2015.10.132,A model with a viable dark matter candidate and massive neutrinos
1,https://openalex.org/W3100627084,https://doi.org/10.1142/s0217732318500244,"Anomalous leptonic U(1) symmetry: Syndetic origin of the QCD axion, weak-scale dark matter, and radiative neutrino mass"
2,https://openalex.org/W3129415996,https://doi.org/10.1103/physrevd.103.095032,Anomaly-free Abelian gauge symmetries with Dirac scotogenic models
3,https://openalex.org/W4206129402,https://doi.org/10.1140/epjc/s10052-021-09918-8,Anomaly-free Abelian gauge symmetries with Dirac seesaws
4,https://openalex.org/W1967455469,https://doi.org/10.1103/physrevd.87.095010,Baryonic violation of R parity from anomalous U(1)H
5,https://openalex.org/W3105890208,https://doi.org/10.1016/s0370-2693(00)01067-4,Bilinear R-parity Violation and Small Neutrino Masses: a Self-consistent Framework
6,https://openalex.org/W2795094497,https://doi.org/10.1103/physrevd.97.115032,Bound-state dark matter and Dirac neutrino masses
7,https://openalex.org/W2963957043,https://doi.org/10.1016/j.physletb.2019.01.023,Bound-state dark matter with Majorana neutrinos
8,https://openalex.org/W2145042168,https://doi.org/10.1140/epjc/s10052-008-0713-4,Collider aspects of flavor physics at high Q
9,https://openalex.org/W2049677811,https://doi.org/10.1088/1126-6708/2005/03/062,Collider signals of gravitino dark matter in bilinearly broken R-parity


In [49]:
f=open('openalexco_A2306595523.json','w')
json.dump(DR,f)
f.close()

In [50]:
f=open('openalexco_A2306595523.json','r')
DR=json.load(f)
f.close()

In [51]:
[x.get('cited_by_count') for x in DR][35:]

[73, 42, 2, 22, 19, 32, 4, 4]

In [52]:
JSON(DR[35])

<IPython.core.display.JSON object>

In [53]:
def order_concepts(l):
    c=[]
    for i in range(5):
        for d in l:
            if d.get('level')==i:
                c.append(f"{d.get('level')}-{d.get('display_name')}")
    return c

In [54]:
pd.set_option('display.max_colwidth',300)

In [55]:
DRdf=pd.DataFrame(DR)

In [56]:
DRdf['c']=DRdf.concepts.apply(order_concepts)

In [57]:
DRdf.drop_duplicates(subset=['title'])[['title','c']]

,title,c
0,Anomaly-free Abelian gauge symmetries with Dirac seesaws,"[0-Physics, 1-Particle physics, 1-Theoretical physics, 2-Anomaly (physics), 2-Homogeneous space, 2-Neutrino, 2-Massless particle, 2-Physics beyond the Standard Model, 2-Symmetry (geometry), 3-Gauge symmetry, 3-Standard Model (mathematical formulation), 4-Seesaw mechanism, 4-Gauge anomaly]"
2,Phenomenology of the Zee model for Dirac neutrinos and general neutrino interactions,"[0-Physics, 1-Particle physics, 2-Neutrino, 2-Phenomenology (philosophy), 2-Physics beyond the Standard Model, 3-Neutrino oscillation, 3-Lepton, 3-Standard Model (mathematical formulation), 3-Dirac (video compression format), 4-Sterile neutrino, 4-Solar neutrino]"
3,Anomaly-free Abelian gauge symmetries with Dirac scotogenic models,"[0-Physics, 1-Particle physics, 1-Theoretical physics, 1-Mathematical physics, 2-Anomaly (physics), 2-Massless particle, 2-Neutrino, 3-Dirac fermion, 3-Gauge symmetry, 3-Standard Model (mathematical formulation), 4-Gauge anomaly]"
4,Phenomenological consistency of the singlet-triplet scotogenic model,"[0-Physics, 1-Particle physics, 1-Nuclear physics, 2-Dark matter, 2-Neutrino, 2-Annihilation, 2-Parameter space, 2-Fermi Gamma-ray Space Telescope, 2-Nucleon, 2-Physics beyond the Standard Model, 3-Lepton, 3-Standard Model (mathematical formulation), 3-Singlet state]"
5,Dirac neutrino mass generation from a Majorana messenger,"[0-Physics, 1-Particle physics, 2-Neutrino, 3-MAJORANA, 3-Dirac (video compression format)]"
6,Métricas de Vinculación Universidad-Entorno: Universidad de Antioquia (Metrics for University Linkage with the Environment: Universidad De Antioquia),"[0-Geography, 1-Humanities, 1-Cartography]"
7,Singlet-doublet Dirac dark matter and neutrino masses,"[0-Physics, 1-Particle physics, 2-Neutrino, 2-Dark matter, 2-Fermion, 3-Dirac (video compression format), 3-Lepton, 3-Neutrino oscillation]"
8,"Two-component dark matter and a massless neutrino in a new <mml:math xmlns:mml=""http://www.w3.org/1998/Math/MathML"" display=""inline""><mml:mi>B</mml:mi><mml:mo>−</mml:mo><mml:mi>L</mml:mi></mml:math> model","[0-Physics, 1-Particle physics, 1-Nuclear physics, 2-Massless particle, 2-Neutrino, 2-Dark matter, 2-Component (thermodynamics)]"
9,Bound-state dark matter with Majorana neutrinos,"[0-Physics, 1-Particle physics, 2-Neutrino, 2-Dark matter, 2-Fermion, 2-Bound state, 3-MAJORANA, 3-Neutrino oscillation, 3-Lepton, 4-Sterile neutrino]"
10,"Probing a simplified W ′ model of R ( D ( * ) ) anomalies using b tags, τ leptons, and missing energy","[0-Physics, 1-Particle physics, 1-Nuclear physics, 2-Quark, 2-Large Hadron Collider, 2-Charm (quantum number), 2-Sensitivity (control systems), 2-Meson, 3-Lepton, 3-Charm quark, 3-B meson, 4-Missing energy]"


In [70]:
any([any( [True,False]), any( [False,False])])

True

In [73]:
ua=[ c for c in co if any([ any([i.get('ror')=="https://ror.org/03bp5hc83" for i in a.get('institutions')]) 
   for a in c.get('authorships')])
]

In [74]:
len(ua)

34291

In [86]:
ph = [ p for p in ua  if any([ c.get('display_name')=='Physics' 
                              or c.get('display_name')=='Materials science'
                              p for c in p.get('concepts')]) ]

In [89]:
len(ph)

3624

In [118]:
l1=[ [c.get('display_name') for c in p.get('concepts') if c.get('level')==1 and eval(c.get('score'))>0.3  ] for p in ph]

In [119]:
l1=[item for sublist in l1 for item in sublist]

In [129]:
dict([(c,l1.count(c)) for c in set(l1) if l1.count(c)>30] ).keys()

dict_keys(['Molecular physics', 'Chemical physics', 'Theoretical physics', 'Inorganic chemistry', 'Particle physics', 'Thermodynamics', 'Humanities', 'Mathematical physics', 'Optoelectronics', 'Astronomy', 'Quantum mechanics', 'Condensed matter physics', 'Crystallography', 'Nanotechnology', 'Chemical engineering', 'Process engineering', 'Statistical physics', 'Pulp and paper industry', 'Composite material', 'Nuclear physics', 'Nuclear engineering', 'Biomedical engineering', 'Optics', 'Astrophysics', 'Atomic physics', 'Metallurgy', 'Polymer chemistry', 'Waste management', 'Organic chemistry', 'Molecular biology', 'Mineralogy', 'Mechanics', 'Nuclear chemistry'])

In [130]:
level1=['Molecular physics', 'Chemical physics','Theoretical physics', 'Particle physics', 
        'Thermodynamics', 'Mathematical physics', 'Optoelectronics', 'Astronomy', 'Quantum mechanics', 'Condensed matter physics', 
        'Crystallography', 'Nanotechnology', 'Statistical physics', 'Composite material', 'Nuclear physics', 
        'Nuclear engineering',  'Optics', 'Astrophysics', 'Atomic physics', 'Mechanics', 'Nuclear chemistry']

In [163]:
df=pd.DataFrame(ph)
dff=df[df['concepts'].apply(lambda l: set([d.get('display_name') for d in l if d.get('level')==1]).intersection(set(level1)) ).apply(len)>0].reset_index(drop=True)

In [215]:
dffo=dff[dff['concepts'].apply(lambda l: any([ d.get('display_name')=='Optics' and d.get('level')==1 for d in l]))]

Go to level 2

In [216]:
lop=dffo.authorships.apply(lambda l: [ a.get('author').get('display_name') for a in l  
                                  if any([i.get('ror')=="https://ror.org/03bp5hc83" for i in a.get('institutions')])   ]   ).to_list()

set([item for sublist in lop for item in sublist ])

{'A. A. Velásquez',
 'A. Herrera',
 'A. Tiutiunnyk',
 'A.J. Smith',
 'Adriana Pabón',
 'Alejandro Mira',
 'Alejandro Mira Agudelo',
 'Alejandro Mira-Agudelo',
 'Alejandro Velez-Zea',
 'Alexis Jaramillo-Osorio',
 'Alvaro Morales',
 'Ana M. Cardenas',
 'Ana Maria Cardenas Soto',
 'Andres Gil Molina',
 'Andrzej Kolodziejczyk',
 'Andrés Felipe Betancur Pérez',
 'B. A. Rodríguez Rey',
 'B. Yepes',
 'Boris A. Rodríguez',
 'Brayan Muñoz',
 'C. A. Barrero',
 'C. A. Betancur-Silvera',
 'C.A. Duque',
 'C.M. Duque',
 'Carlos Andres Vera',
 'Carlos Andrés Vargas',
 'Carlos Vargas',
 'César Restrepo',
 'D Aristizábal-Giraldo',
 'D Muñetón',
 'D. C. Alvarez-Palacio',
 'D. E. Jaramillo',
 'Daniel H. Martínez-Suárez',
 'Daniel Sánchez',
 'Diana Margarita',
 'E. Reyes-Gómez',
 'E. Tangarife',
 'Edgar Rueda',
 'Edwar A. Castaneda Zapata',
 'Edward Hernández',
 'F. Arango',
 'F. E. Lopez',
 'F. Gómez',
 'F. Londoño',
 'Fabián E. Cano Ardila',
 'Felipe Ramirez',
 'Fernando A. Londono',
 'Fernando Medina',

In [226]:
l2=dffo.concepts.apply( lambda p:  [c.get('display_name') for c in p if c.get('level')==2 and eval(c.get('score'))>0.3  ]     ).to_list()
l2=set([item for sublist in l2 for item in sublist])

In [229]:
level2={'Absorbance',
 'Absorption (acoustics)',
 'Absorption spectroscopy',
 'Access network',
 'Achromatic lens',
 'Adaptive optics',
 'Angle of incidence (optics)',
 'Astigmatism',
 'Attenuation coefficient',
 'Beam (structure)',
 'Binocular vision',
 'Birefringence',
 'Brillouin zone',
 'Broadband',
 'Calibration',
 'Channel (broadcasting)',
 'Chromatic scale',
 'Chromaticity',
 'Coercivity',
 'Coherence (philosophical gambling strategy)',
 'Collimator',
 'Color measurement',
 'Compression (physics)',
 'Confocal microscopy',
 'Contrast (vision)',
 'Curvature',
 'Decoding methods',
 'Depth of field',
 'Dielectric',
 'Diffraction',
 'Digital micromirror device',
 'Diode',
 'Dispersion (optics)',
 'Dispersion relation',
 'Elastic modulus',
 'Elasticity (physics)',
 'Electric field',
 'Electrical conductor',
 'Electrical resistivity and conductivity',
 'Electrode',
 'Electromagnetic field',
 'Electromagnetic radiation',
 'Electron',
 'Electronic band structure',
 'Electronic structure',
 'Encoding (memory)',
 'Encryption',
 'Energy (signal processing)',
 'Energy flux',
 'Fiber',
 'Field of view',
 'Filter (signal processing)',
 'Finite element method',
 'Flux (metallurgy)',
 'Focus (optics)',
 'Footprint',
 'Fourier transform',
 'Frame (networking)',
 'Frame rate',
 'Full width at half maximum',
 'Gaussian',
 'Geometrical optics',
 'Ghost imaging',
 'Glass fiber',
 'Graphene',
 'Graphite',
 'Grating',
 'Holography',
 'Image (mathematics)',
 'Image resolution',
 'Imaging phantom',
 'Impulse response',
 'Integrated optics',
 'Intensity (physics)',
 'Interferometry',
 'Iridescence',
 'Isotropy',
 'Keying',
 'Kinetic energy',
 'Lamina',
 'Laser',
 'Lattice (music)',
 'Lens (geology)',
 'Light beam',
 'Light field',
 'Light intensity',
 'Liquid crystal',
 'Magnetic field',
 'Microscope',
 'Microscopy',
 'Microwave',
 'Modulation (music)',
 'Moiré pattern',
 'Monochromatic color',
 'Monochromatic electromagnetic plane wave',
 'Monocular',
 'Monocular vision',
 'Monolayer',
 'Morse code',
 'Morse potential',
 'Multiplex',
 'Multiplexing',
 'Multiplier (economics)',
 'Nanoindentation',
 'Nanomaterials',
 'Near and far field',
 'Nonimaging optics',
 'Nonlinear system',
 'Opacity',
 'Optical coherence tomography',
 'Optical communication',
 'Optical engineering',
 'Optical fiber',
 'Optical filter',
 'Optical materials',
 'Optical switch',
 'Parabolic reflector',
 'Parametric statistics',
 'Particulates',
 'Path (computing)',
 'Phase (matter)',
 'Phonon',
 'Photodetector',
 'Photon',
 'Photonic crystal',
 'Photonics',
 'Photorefractive effect',
 'Photothermal therapy',
 'Physical optics',
 'Pinhole (optics)',
 'Pixel',
 'Plane (geometry)',
 'Polarization (electrochemistry)',
 'Polarized light microscopy',
 'Power (physics)',
 'Presbyopia',
 'Pupil',
 'Radio frequency',
 'Radio telescope',
 'Raman spectroscopy',
 'Randomness',
 'Ray',
 'Reflection (computer programming)',
 'Reflectivity',
 'Refraction',
 'Refractive index',
 'Resolution (logic)',
 'SIGNAL (programming language)',
 'Scattering',
 'Sensitivity (control systems)',
 'Simulated annealing',
 'Slit',
 'Spatial correlation',
 'Spatial frequency',
 'Spatial light modulator',
 'Speckle pattern',
 'Spectral density',
 'Spectral line',
 'Spectrometer',
 'Spectrophotometry',
 'Spectroscopy',
 'Spectrum (functional analysis)',
 'Specular reflection',
 'Spiral (railway)',
 'Tears',
 'Transfer function',
 'Transfer matrix',
 'Transfer-matrix method (optics)',
 'Transmission (telecommunications)',
 'Transmittance',
 'Transverse plane',
 'Vibration',
 'Visibility',
 'Visual inspection',
 'Vortex',
 'Wave propagation',
 'Wavefront',
 'Waveguide',
 'Wavelength',
 'White light',
 'Window (computing)'
}

In [231]:
dffo=dffo[dffo['concepts'].apply(lambda l: set([d.get('display_name') for d in l if d.get('level')==2]).intersection(set(level2)) ).apply(len)>0].reset_index(drop=True)

In [243]:
cl=dffo.concepts.apply(lambda l: [(d.get('display_name'),d.get('score')) for d in l if d.get('level')==2]).to_list()
dict([item for subitem in cl for item in subitem])

{'Holography': '0.4111298',
 'Vortex': '0.7507218',
 'Track (disk drive)': '0.66748726',
 'Tracking (education)': '0.5897759',
 'Phase (matter)': '0.6633542',
 'Object (grammar)': '0.57633126',
 'Displacement (psychology)': '0.370496',
 'Halide': '0.8800218',
 'Perovskite (structure)': '0.8272786',
 'Iodide': '0.6832401',
 'Thin film': '0.5584189',
 'Synchrotron': '0.547082',
 'Microstructure': '0.5384922',
 'Strain (injury)': '0.5371339',
 'Diffraction': '0.6714398',
 'Synchrotron radiation': '0.41056058',
 'Reflectivity': '0.7579099',
 'Composition (language)': '0.5789246',
 'Spectral line': '0.49719217',
 'Hue': '0.39289743',
 'Analytical Chemistry (journal)': '0.4899737',
 'Spectroscopy': '0.47463238',
 'Spectrophotometry': '0.47897008',
 'Elemental analysis': '0.43234447',
 'Spatial light modulator': '0.369657',
 'Amplitude': '0.48458707',
 'Constraint (computer-aided design)': '0.5336987',
 'Plane (geometry)': '0.52911896',
 'Projection (relational algebra)': '0.352074',
 'Digita

In [ ]:
[ [c.get('display_name') for c in p.get('concepts') if c.get('level')==1 and eval(c.get('score'))>0.3  ] for p in ph]

In [217]:
dffo[dffo.authorships.apply(lambda l: any([ a.get('author').get('display_name')=='Alvaro Morales' for a in l]) )].concepts.iloc[0]

[{'id': 'https://openalex.org/C2910081258',
  'wikidata': 'https://www.wikidata.org/wiki/Q177807',
  'display_name': 'Hydrostatic pressure',
  'level': 2,
  'score': '0.7969276'},
 {'id': 'https://openalex.org/C520434653',
  'wikidata': 'https://www.wikidata.org/wiki/Q38867',
  'display_name': 'Laser',
  'level': 2,
  'score': '0.55297494'},
 {'id': 'https://openalex.org/C29169072',
  'wikidata': 'https://www.wikidata.org/wiki/Q521166',
  'display_name': 'Quantum well',
  'level': 3,
  'score': '0.52323693'},
 {'id': 'https://openalex.org/C121332964',
  'wikidata': 'https://www.wikidata.org/wiki/Q413',
  'display_name': 'Physics',
  'level': 0,
  'score': '0.51779354'},
 {'id': 'https://openalex.org/C207467116',
  'wikidata': 'https://www.wikidata.org/wiki/Q4385666',
  'display_name': 'Inverse',
  'level': 2,
  'score': '0.4749715'},
 {'id': 'https://openalex.org/C15708023',
  'wikidata': 'https://www.wikidata.org/wiki/Q80083',
  'display_name': 'Humanities',
  'level': 1,
  'score': '

In [214]:
dffo[dffo.authorships.apply(lambda l: any([ a.get('author').get('display_name')=='B. A. Rodríguez Rey' for a in l]) )].iloc[0].concepts

[{'id': 'https://openalex.org/C136479403',
  'wikidata': 'https://www.wikidata.org/wiki/Q1047683',
  'display_name': 'Quasiparticle',
  'level': 3,
  'score': '0.80569255'},
 {'id': 'https://openalex.org/C84114770',
  'wikidata': 'https://www.wikidata.org/wiki/Q46344',
  'display_name': 'Quantum',
  'level': 2,
  'score': '0.5456141'},
 {'id': 'https://openalex.org/C124657808',
  'wikidata': 'https://www.wikidata.org/wiki/Q1133068',
  'display_name': 'Quantum dot',
  'level': 2,
  'score': '0.42620873'},
 {'id': 'https://openalex.org/C121332964',
  'wikidata': 'https://www.wikidata.org/wiki/Q413',
  'display_name': 'Physics',
  'level': 0,
  'score': '0.38675803'},
 {'id': 'https://openalex.org/C120665830',
  'wikidata': 'https://www.wikidata.org/wiki/Q14620',
  'display_name': 'Optics',
  'level': 1,
  'score': '0.36696696'}]

https://www.propelauth.com/post/autogenerating-clients-with-fastapi-and-github-actions